In [1]:
import cv2
import numpy as np
import dlib
from math import hypot

In [2]:
# Loading Camera and eye image and Creating mask
#cap = cv2.VideoCapture(0) #Load Camera Video
cap = cv2.VideoCapture(0)
eye_image = cv2.imread('censor.png')
_, frame = cap.read()
rows, cols, _ = frame.shape
eye_mask = np.zeros((rows, cols), np.uint8)

In [3]:
# Loading Face detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Explain what is the dlib shape predictor

In [4]:
while True:
    _, frame = cap.read()
    eye_mask.fill(0)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(frame)
    for face in faces:
        landmarks = predictor(gray_frame, face)

        # Eye coordinates
        top_eye = (landmarks.part(19).x, landmarks.part(24).y) #previously 29
        center_eye = (landmarks.part(30).x, landmarks.part(30).y)
        left_eye = (landmarks.part(0).x, landmarks.part(0).y) #31
        right_eye = (landmarks.part(16).x, landmarks.part(16).y) #35

        eye_width = int(hypot(left_eye[0] - right_eye[0],
                           left_eye[1] - right_eye[1]) * 1.7)
        eye_height = int(eye_width * 0.25) #0.77

        # New eye position
        top_left = (int(center_eye[0] - eye_width / 2), #(int(center_eye[0] - eye_width / 2)
                              int(center_eye[1] - eye_height / 0.8))# 2
        bottom_right = (int(center_eye[0] + eye_width / 2),
                       int(center_eye[1] + eye_height / 2))


        # Adding the new eye
        eye_bar = cv2.resize(eye_image, (eye_width, eye_height))
        eye_bar_gray = cv2.cvtColor(eye_bar, cv2.COLOR_BGR2GRAY)
        _, eye_mask = cv2.threshold(eye_bar_gray, 25, 255, cv2.THRESH_BINARY_INV)

        eye_area = frame[top_left[1]: top_left[1] + eye_height,
                    top_left[0]: top_left[0] + eye_width]
        eye_area_no_eye = cv2.bitwise_and(eye_area, eye_area, mask=eye_mask)
        final_eye = cv2.add(eye_area_no_eye, eye_bar)

        frame[top_left[1]: top_left[1] + eye_height,
                    top_left[0]: top_left[0] + eye_width] = final_eye

        cv2.imshow("eye area", eye_area)
        cv2.imshow("eye bar", eye_bar)
        cv2.imshow("final eye", final_eye)



    cv2.imshow("Frame", frame) # Display video



    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        break

In [5]:
        cap.release()
        cv2.destroyAllWindows()